In [3]:
sqls = ["SELECT MIN(points) FROM DF WHERE rider = 'roger dutton / tony wright';"]
thoughts = ["Based on the SQL query provided, the minimum number of points that Roger Dutton / Tony Wright received in the 1972 Isle of Man TT event was 3. Therefore, the claim that 2 is the fewest points they received is false. The output should be 0."]
tables = ["<table>\n<caption>1972 isle of man tt</caption>\n<thead>\n<tr><th>  MIN(points)</th></tr>\n</thead>\n<tbody>\n<tr><td>3            </td></tr>\n</tbody>\n</table>"]
claims = ["2 be the fewest point that roger dutton / tony wright receive"]
dict(zip(["SQL", "table", "claim", "thought"], [sqls[0], tables[0], claims[0], thoughts[0]]))

{'SQL': "SELECT MIN(points) FROM DF WHERE rider = 'roger dutton / tony wright';",
 'table': '<table>\n<caption>1972 isle of man tt</caption>\n<thead>\n<tr><th>  MIN(points)</th></tr>\n</thead>\n<tbody>\n<tr><td>3            </td></tr>\n</tbody>\n</table>',
 'claim': '2 be the fewest point that roger dutton / tony wright receive',
 'thought': 'Based on the SQL query provided, the minimum number of points that Roger Dutton / Tony Wright received in the 1972 Isle of Man TT event was 3. Therefore, the claim that 2 is the fewest points they received is false. The output should be 0.'}

In [19]:
# transform data into sqlite database
from data_loader import TableLoader

table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)




In [2]:
from datasets import load_dataset
totto = load_dataset('./data_loader/totto_zh.py', verification_mode="no_checks", cache_dir="/media/disk2/datasets")
tabfact = load_dataset('./data_loader/tabfact.py', verification_mode="no_checks", cache_dir="/media/disk2/datasets")

In [34]:
filtered_dataset = table_loader.dataset.filter(lambda example: example['small_test'])

Filter: 100%|██████████| 1000/1000 [00:00<00:00, 4669.79 examples/s]


In [35]:
filtered_dataset

Dataset({
    features: ['id', 'table', 'statement', 'label', 'hardness', 'small_test'],
    num_rows: 147
})

In [13]:
len(table_loader.dataset)


47

In [18]:
test_sample = table_loader.dataset[2]
test_sample

{'id': 996,
 'table': {'id': '2-16369528-1.html.csv',
  'header': ['year',
   'best teenage / young adult',
   "reader 's vote",
   'best non - fiction',
   'lifetime achievement'],
  'rows': [['1996', 'na', 'na', 'na', 'jon cleary'],
   ['1997',
    'na',
    'na',
    'how to write crime edited by marele day',
    'alan yates (aka carter brown )'],
   ['1998', 'na', 'na', 'na', 'na'],
   ['1999', 'na', 'na', 'na', 'peter corris'],
   ['2000', 'na', 'na', 'na', 'na'],
   ['2001',
    'na',
    'bleeding hearts by lindy cameron',
    'na',
    'professor stephen knight'],
   ['2002',
    'blue murder by ken catran',
    'apartment 255 by bunty avieson',
    'na',
    'patrick gallagher'],
   ['2003', 'na', 'na', 'na', 'kerry greenwood'],
   ['2004', 'na', 'na', 'na', 'bob bottom'],
   ['2005', 'na', 'na', 'na', 'stuart coupe'],
   ['2006', 'na', 'na', 'na', 'andrew rule and john silvester'],
   ['2007', 'na', 'na', 'na', 'sandra harvey and lindsay simpson'],
   ['2008', 'na', 'na', 'na

### Data Augmentation

In [ ]:
#引入 schema(table measurement) statistical(numerical) enum string(char) date
#引入 term explanation（table comment(RAG)）
#引入 column summarization
#是否需要search engine
#table size, statistical features, header hierarchy


In [14]:
#schema 
from data_loader import TableFormat
from langchain.prompts.prompt import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI, OpenAI
model = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_base="https://api.chatanywhere.tech/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")
formatter = TableFormat(format='none', data=test_sample, use_sampling=True)
pre_instruction = PromptTemplate(input_variables=["table"], template=
"""
Instruction: Given the following table, you will add Metadata about the columns in the table.
Metadata includes:
- Numerical: whether the column content is numeric type like int or float.
- Char: whether the column content is a text or description.
- Date: whether the column content is datetime.

You need to output all the column names with metadata in angle brackets.
Example: name<Char>, launched<Date>, count<Numerical>

Table: {table}
Output:
""")
# 
# output = model.invoke([HumanMessage(content=pre_instruction.format(table=formatter.format_html()))])

In [16]:
output.response_metadata['token_usage']

{'completion_tokens': 1, 'prompt_tokens': 269, 'total_tokens': 270}

In [13]:
#summarization
pre_instruction_summary = PromptTemplate(input_variables=['table'], template=
"""
Instruction: Given the following table, you need to first summarize the contents of the table, then based on the summay, give a concluded description to each of the column.
Table: {table}

The output should use the following format: 
table summary: #summary for table contents
column description: You need to output all the column names with description in angle brackets
example: launched<The launched date for the competition> date<The date of the match>
""")
output = model.invoke([HumanMessage(content=pre_instruction_summary.format(table=formatter.format_html()))])

APIConnectionError: Connection error.

In [12]:
print(output.content)

table summary: This table contains information about tennis matches including the date, tournament, surface, partner, opponents, and score.

column description: 
date<The date of the match>
tournament<The name and location of the tournament>
surface<The type of surface the match was played on>
partner<The name of the player's doubles partner>
opponents<The names of the opposing doubles team>
score<The final score of the match, including sets won by each team>


In [11]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = "true"

In [10]:
#table_size
def get_table_size():
    return f'The table has {formatter.data.shape[0]} rows and {formatter.data.shape[1]} columns.'
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_base="https://api.chatanywhere.cn/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW", temperature=0.1)
from data_loader import TableAug,TableFormat
aug = TableAug(model)
# aug.schema_aug(test_sample)
test_sample = table_loader.dataset[0]
formatter = TableFormat(format='none')
output = aug.batch_summary_aug(formatter, table_loader.dataset[:2], batch_size=2, output_token=True)
# batch_data = table_loader.dataset[:2]

# print([formatter.load_data_from_dic({key: value[i] for key, value in batch_data.items()}).format_html() for i in range(len(batch_data.keys())) ])
# print([formatter.load_data_from_dic({key: value[i] for key, value in table_loader.dataset[:2].items()}).format_html() for i in range(2)])

KeyError: 0

In [9]:
print(output[0])

{'text': 'table summary: The table provides information about different wind farms including their names, scheduled dates, capacity in megawatts, number of turbines, types of turbines, and locations.\n\ncolumn description: \nwind farm<The name of the wind farm>\nscheduled<The scheduled date for the wind farm>\ncapacity (mw)<The capacity of the wind farm in megawatts>\nturbines<The number of turbines at the wind farm>\ntype<The type of turbines used at the wind farm>\nlocation<The location of the wind farm>'}


In [12]:
formatter = TableFormat(format='none',data=test_sample)
aug.summary_aug(formatter)

("Golf tournament winners' information",
 "player<The name of the player>\ncountry<The country the player represents>\nyear (s) won<The year(s) the player won the tournament>\ntotal<The total score of the player>\nto par<The score in relation to par (under par or over par)>\nfinish<The player's finishing position in the tournament>")

### Few Shot learning

In [9]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from data_loader import TableFormat
summary_examples = ['The columns in the table are "wind farm, scheduled, capacity (mw), turbines, type, and location." The rows in the table represent different wind farms, with information about their scheduled dates, capacity, number of turbines, type, and location.',
                    "The table provides information about different destroyers, including their kanji (Japanese characters), names, builders, dates when they were laid down, launched, and completed."]
subtable_examples = ['Columns(wind farm, scheduled, turbines), Rows(12)',
                    'Columns(kanji, builder), Rows(ALL)']
inds = [0, 100]
examples_prompt = PromptTemplate(input_variables=["table", "claim", "summary", "subtable"], template=
"""
Table: {table}
Claim: {claim}
Summary: {summary}
Subtable: {subtable}""")
formatter = TableFormat(format='none', data=test_sample, use_sampling=True)
num_k = 2
# examples = [examples_prompt.format(**{"table": TableFormat(format='none', data=table_loader.dataset[inds[i]]).format_html(),
#                                     "claim": table_loader.dataset[inds[i]]['statement'],
#                                     "summary": summary_examples[i],
#                                     "subtable": subtable_examples[i]}) for i in range(num_k)]

examples_dict = [{"table": TableFormat(format='none', data=table_loader.dataset[inds[i]], use_sampling=True).format_html(),
                                    "claim": table_loader.dataset[inds[i]]['statement'],
                                    "summary": summary_examples[i],
                                    "subtable": subtable_examples[i]} for i in range(num_k)]
prompt = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="Instruction: Given the following table and claim, let's first summarize the contents of the rows and columns of the table, and then select relevent rows/columns in the given table that support or oppose the statement.",
    suffix=
    """
Table: {table}
Claim: {claim}
    """,
    input_variables=["table", "claim"],
)
print(prompt.format(table=formatter.format_html(), claim=test_sample['statement']))

Instruction: Given the following table and claim, let's first summarize the contents of the rows and columns of the table, and then select relevent rows/columns in the given table that support or oppose the statement.


Table: <table>
<thead>
<tr><th>  wind farm</th><th>  scheduled</th><th>  capacity (mw)</th><th>  turbines</th><th>              type</th><th>      location</th></tr>
</thead>
<tbody>
<tr><td>codling    </td><td>unknown    </td><td>1100           </td><td>220       </td><td>unknown           </td><td>county wicklow</td></tr>
<tr><td>carrowleagh</td><td>2012       </td><td>36.8           </td><td>16        </td><td>enercon e - 70 2.3</td><td>county cork   </td></tr>
<tr><td>gortahile  </td><td>2010 autumn</td><td>20             </td><td>8         </td><td>nordex n90        </td><td>county laois  </td></tr>
</tbody>
</table>
Claim: the scheduled date for the farm with 17 turbine be 2012
Summary: The columns in the table are "wind farm, scheduled, capacity (mw), turbines, t

In [5]:
from data_loader import TableFormat
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.messages import HumanMessage
from langchain.prompts.prompt import PromptTemplate
model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_base="https://api.chatanywhere.cn/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")
model.invoke([HumanMessage(content=prompt.format(table=formatter.format_html(), claim=test_sample['statement']))])

AIMessage(content='Summary: The table provides information about different events, including their names, establishment years, categories, subcategories, and main venues.\nSubtable: Columns(event name, established, category), Rows(2)', response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 931, 'total_tokens': 972}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None})

### Few-shot for operations

In [38]:
from data_loader import TableFormat
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.messages import HumanMessage
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
model = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_base="https://api.chatanywhere.cn/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")
formatter = TableFormat(format='none', data=test_sample, use_sampling=True)
inds = [0,2]
summary_examples = ['The columns in the table are "wind farm, scheduled, capacity (mw), turbines, type, and location." The rows in the table represent different wind farms, with information about their scheduled dates, capacity, number of turbines, type, and location.',
                    "The table provides information about different events, including their names, establishment years, categories, subcategories, and main venues."]
Output_examples = ['wind farm<DELETE>, scheduled<KEEP>, capacity (mw)<DELETE>, turbines<KEEP>, type<KEEP>, location<KEEP>', 'event name<KEEP> established<KEEP> category<KEEP> sub category<DELETE> main venue<DELETE>']
examples_prompt = PromptTemplate(input_variables=["table", "claim", "output"], template=
"""
Table: {table}
Claim: {claim}
Output: {output}""")
num_k = 2
examples_dict = [{"table": TableFormat(format='none', data=table_loader.dataset[inds[i]], use_sampling=True).format_html(),
                                    "claim": table_loader.dataset[inds[i]]['statement'],
                                    # "summary": summary_examples[i],
                                    "output": Output_examples[i]} for i in range(num_k)]
prompt = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="""You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
Instruction: Given the following table and claim, you will output the operations corresponding to each column which can help us judging the truth or falsity of claim.
Operations: DELETE(delete column unrelevant to the claim), KEEP(keep column relevant to the claim), GROUP BY(combine aggregate functions and group the result set by one or more columns), COUNT(returns the number of rows in column), AVG(returns the average value of a numeric column), SUM(returns the sum of a numeric column), MAX(returns the max value of a numeric column), MIN(returns the min value of a numeric column), ORDER BY(sort the value in ascending order)""",
    suffix=
    """
Table: {table}
Thought: your reason here
Claim: {claim}
    """,
    input_variables=["table", "claim"],
)
print(prompt.format(table=formatter.format_html(), claim=test_sample['statement']))

# output = model.invoke([HumanMessage(content=prompt.format(table=formatter.format_html(), claim=test_sample['statement']))])



You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
Instruction: Given the following table and claim, you will output the operations corresponding to each column which can help us judging the truth or falsity of claim.
Operations: DELETE(delete column unrelevant to the claim), KEEP(keep column relevant to the claim), GROUP BY(combine aggregate functions and group the result set by one or more columns), COUNT(returns the number of rows in column), AVG(returns the average value of a numeric column), SUM(returns the sum of a numeric column), MAX(returns the max value of a numeric column), MIN(returns the min value of a numeric column), ORDER BY(sort the value in ascending order)


Table: <table>
<thead>
<tr><th>  wind farm</th><th>  scheduled</th><th>  capacity (mw)</th><th>  turbines</th><th>              type</th><th>      location</th></tr>
<

In [39]:
output = model.invoke([HumanMessage(content=prompt.format(table=formatter.format_html(), claim=test_sample['statement']))])
print(output)

content='Output: date<DELETE>, tournament<DELETE>, surface<DELETE>, partner<KEEP>, opponents<KEEP>, score<DELETE>' response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 1037, 'total_tokens': 1063}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}


In [35]:
print(formatter.format_psql())
print(test_sample['statement'])
print(test_sample['label'])

+------------------+-----------------------------------+-----------+---------------------+------------------------------------------------+-----------------------+
| date             | tournament                        | surface   | partner             | opponents                                      | score                 |
|------------------+-----------------------------------+-----------+---------------------+------------------------------------------------+-----------------------|
| 15 january 2007  | algiers 2 , algeria               | clay      | rushmi chakravarthi | barbora matusova anna savitskaya               | 6 - 2 , 6 - 0         |
| 11 february 2008 | mallorca 2 , spain                | clay      | stephanie vogt      | leticia costas - moreira maite gabarrus alonso | 7 - 6 (7 - 2) , 6 - 3 |
| 28 april 2008    | makarska , croatia                | clay      | stephanie vogt      | tadeja majerić maša zec peškirič               | 7 - 5 , 6 - 2         |
| 8 september 20

## Zero-shot learning


In [22]:
from tabulate import tabulate
formatter = TableFormat(format='none', data=test_sample)
print(test_sample['statement'])
print(tabulate(formatter.data, headers=formatter.data.columns, tablefmt='psql'))
# print('Summary: The table provides information about different events, including their names, establishment years, categories, subcategories, and main venues.\nSubtable: Columns(event name, established, category), Rows(2)')

uraga dock company produce the most destroyer on the list , at 3
+----+---------+-------------------+--------------------------------+------------------+------------------+-------------------+
|    | kanji   | name              | builder                        | laid down        | launched         | completed         |
|----+---------+-------------------+--------------------------------+------------------+------------------+-------------------|
|  0 | 睦月    | mutsuki dd - 19   | sasebo naval arsenal , japan   | 21 may 1924      | 23 july 1925     | 25 march 1926     |
|  1 | 如月    | kisaragi dd - 21  | maizuru naval arsenal , japan  | 3 june 1924      | 5 june 1925      | 21 december 1925  |
|  2 | 彌生    | yayoi dd - 23     | uraga dock company , japan     | 11 january 1924  | 11 july 1925     | 28 august 1926    |
|  3 | 卯月    | uzuki dd - 25     | ishikawajima shipyards , japan | 11 january 1924  | 15 october 1925  | 14 september 1926 |
|  4 | 皐月    | satsuki dd - 27   | fujinagata s

In [11]:
# for sample in table_loader.dataset:
#     print(sample)
#     test_sample = sample
#     break

from data_loader import TableFormat
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.messages import HumanMessage
from langchain.prompts.prompt import PromptTemplate
model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_base="https://api.chatanywhere.cn/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")
formatter = TableFormat(format='none', data=test_sample, use_sampling=True)
inds = [0, 100]
Output_examples = ['event name(KEEP) established(KEEP) category(KEEP) sub category(DELETE) main venue(DELETE)', 'wind farm, scheduled, capacity (mw), turbines, type, and location.']
pre_instruction = PromptTemplate(input_variables=["table", "claim"], template=
"""
Instruction: Given the following table and claim, let's first summarize the contents of the table, and then output the operations corresponding to each column which can help us judging the truth or falsity of claim.
Operations: DELETE, KEEP, GROUP BY, COUNT, AVG, SUM, MAX, MIN, ORDER BY

Table: {table}
Claim: {claim}
Output:
""")
# 
output = model.invoke([HumanMessage(content=pre_instruction.format(table=formatter.format_html(), claim=test_sample['statement']))])



In [13]:
print(output.content)

Summary of the contents of the rows and columns of the table:

Rows:
1. Event name: The Frye Festival
   Established: 2000
   Category: Arts
   Sub category: Literary
   Main venue: University of Moncton

2. Event name: World Wine & Food Expo
   Established: 1990
   Category: Arts
   Sub category: Food & Drink
   Main venue: Moncton Coliseum

3. Event name: Dieppe Kite International
   Established: 2001
   Category: Sporting
   Sub category: Kite Flying
   Main venue: Dover Park

Columns:
1. Event name
2. Established
3. Category
4. Sub category
5. Main venue

Operations corresponding to each column that can help us judge the truth or falsity of the claim:

1. Event name: No specific operation is necessary for this column to judge the claim.
2. Established: We can use the operation MIN to find the earliest establishment year.
3. Category: We can use the operation GROUP BY to group events by category and then compare if the given category exists.
4. Sub category: No specific operation is

In [40]:
from tabulate import tabulate
print(tabulate(formatter.data, headers=formatter.data.columns, tablefmt='psql'))
print('Summary: The table provides information about different events, including their names, establishment years, categories, subcategories, and main venues.\nSubtable: Columns(event name, established, category), Rows(touchdown atlantic)')

+----+--------------------------------------------+---------------+----------------+----------------+--------------------------+
|    | event name                                 |   established | category       | sub category   | main venue               |
|----+--------------------------------------------+---------------+----------------+----------------+--------------------------|
|  0 | dieppe kite international                  |          2001 | sporting       | kite flying    | dover park               |
|  1 | the frye festival                          |          2000 | arts           | literary       | university of moncton    |
|  2 | hubcap comedy festival                     |          2000 | arts           | comedy         | various                  |
|  3 | touchdown atlantic                         |          2010 | sporting       | football       | moncton stadium          |
|  4 | atlantic nationals automotive extravaganza |          2000 | transportation | automotive  

In [41]:
print(test_sample['statement'])

touchdown atlantic , in the category of sporting , be establish in 2010


In [38]:
import os
from tenacity import retry, stop_after_attempt, wait_random_exponential
from typing import List
from openai import OpenAI
class Config:
    def __init__(self) -> None:
        self.openai_api_key = "sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW"
        self.base_url = "https://api.chatanywhere.cn/v1"
        self.model = "gpt-3.5-turbo"

class CallLLM:
    """Class for calling the OpenAI Language Model API."""

    def __init__(self, config: Config):
        self.client = OpenAI(api_key=config.openai_api_key,
                             base_url=config.base_url)
        self.model = config.model
        
    @retry(wait=wait_random_exponential(min=30, max=60), stop=stop_after_attempt(1000))
    def generate_text(self, prompt: List[str]) -> List[str]:
        """Generate text based on the prompt and instruction."""

            # batched examples, with xx completions per request
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}], 
            temperature=0,
            max_tokens=96,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0,
        )
        # match completions to prompts by index
        return response.choices[0].message.content.strip()
configs = Config()  
llm = CallLLM(config=configs)


In [39]:

input_text = "\n".join(input)
llm.generate_text(input_text)

'1'

In [15]:
import pandas as pd
def process_input(sample, instruction):
    df = pd.DataFrame(columns=sample["table"]["header"])
    for i, line in enumerate(sample['table']['rows']):
        df.loc[i] = line
        
    texts = [instruction,
          "The database table DF is shown as follows: \n",
          df.to_html(),
          "query:",
          sample['statement'],
           "Output the code braced by '```'. \n SQL:"]
    
    label = sample['label']
    return "\n".join(texts), label 
    

In [4]:
subtable_instruction = """
You are a SQLite expert. Given an input query, identify critical values and ranges of the table, then create a syntactically correct SQLite query to create a VIEW. To create a syntactically correct SQL view, the selected data within this view must be helpful in answering the question. During the construction of the view, if column names are confusing, rename the columns accordingly.
"""

In [16]:
print(process_input(test_sample, subtable_instruction))

('\nYou are a SQLite expert. Given an input query, identify critical values and ranges of the table, then create a syntactically correct SQLite query to create a VIEW. To create a syntactically correct SQL view, the selected data within this view must be helpful in answering the question. During the construction of the view, if column names are confusing, rename the columns accordingly.\n\nThe database table DF is shown as follows: \n\n<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>date</th>\n      <th>result</th>\n      <th>score</th>\n      <th>brazil scorers</th>\n      <th>competition</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>may 11 , 1919</td>\n      <td>w</td>\n      <td>6 - 0</td>\n      <td>friedenreich (3) , neco (2) , haroldo</td>\n      <td>south american championship</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>may 18 , 1919</td>\n      <td>w</td>\n      <td>3 - 1

### Simple pipeline

In [6]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine('sqlite:///db/sqlite/tabfact.db')
normalized = table_loader.normalize_table(test_sample)
df = pd.DataFrame(columns=normalized['table']['header'])
for ind, r in enumerate(normalized['table']['rows']):
    df.loc[ind] = r
# print(' '.join(normalized['table']['header']) + '*************')
df.to_sql(name='ind100', con=engine, if_exists='replace', index=False)
# table_loader.table2db(engine, test_sample)

12

In [34]:
from tabulate import tabulate
structured_data_markdown = tabulate(
            df, headers=df.columns, tablefmt="pipe", showindex=True
        )

In [35]:
structured_data_markdown

'|    | date          | result   | score   | brazil scorers                        | competition                 |\n|---:|:--------------|:---------|:--------|:--------------------------------------|:----------------------------|\n|  0 | may 11 , 1919 | w        | 6 - 0   | friedenreich (3) , neco (2) , haroldo | south american championship |\n|  1 | may 18 , 1919 | w        | 3 - 1   | heitor , amílcar , millon             | south american championship |\n|  2 | may 26 , 1919 | d        | 2 - 2   | neco (2)                              | south american championship |\n|  3 | may 29 , 1919 | w        | 1 - 0   | friedenreich                          | south american championship |\n|  4 | june 1 , 1919 | d        | 3 - 3   | haroldo , arlindo (2)                 | taça roberto cherry         |'

In [1]:

from langchain_openai import ChatOpenAI, OpenAI
model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_base="https://api.chatanywhere.cn/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")
model = ChatOpenAI(model_name="gpt-4", openai_api_base="https://api.chatanywhere.cn/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")

In [36]:
import re
def extract_yes_no_and_map(text):
    # Convert the input text to lowercase for case-insensitive matching
    text = text.lower()

    # Define regular expressions for yes/no matching
    yes_patterns = [r'\byes\b', r'\btrue\b']
    no_patterns = [r'\bno\b', r'\bfalse\b']

    # Check for "0"
    if text == "0":
        return "0"

    # Check for "1"
    if text == "1":
        return "1"

    # Check for yes
    for pattern in yes_patterns:
        if re.search(pattern, text):
            return "1"

    # Check for no
    for pattern in no_patterns:
        if re.search(pattern, text):
            return "0"

    # Return 2 if neither yes nor no is found
    return "2"

def eval_fv_match(pred_list, gold_list):
        acc = 0.0
        for pred, gold in zip(pred_list, gold_list):
            pred, gold = extract_yes_no_and_map(pred), extract_yes_no_and_map(gold)
            if pred == gold:
                acc += 1
        acc = acc / len(pred_list)
        return acc
    
def process_input(sample, instruction):
    df = pd.DataFrame(columns=sample["table"]["header"])
    for i, line in enumerate(sample['table']['rows']):
        df.loc[i] = line
        
    texts = [instruction,
          "the table needed to be answered: \n",
          df.to_html(),
          "query:",
          sample['statement'],
           "answer is: \n"]
    
    label = sample['label']
    return "\n".join(texts), label 

In [37]:
#简易版本的pipeline
instruction = "Read the table below to verify whether the provided claim/query are true or false. Return 0 if it's false, or 1 if it's true. Only return 0 or 1 without any other information. \n"
num_samples = 32
batch_size = 32
num_batches = num_samples // batch_size
batches = []
for batch_num in range(num_batches):
    batch_prompt, ground = [], []
    
    start = batch_num * batch_size
    batch_data = tabfact['validation'][start: start+batch_size]
    for i in range(batch_size):
        prompt, label = process_input(sample=dict({key: value[i] for key, value in batch_data.items()}), instruction=instruction)
        batch_prompt.append(prompt)
        ground.append(str(label))
        # call llm
    batch_pred = list(map(lambda x: x.content, model.batch(batch_prompt)))
        
        #do evaluation
    accuracy = eval_fv_match(batch_pred, ground)
    print(accuracy)
        
        
        
    

        
        

0.6875


In [42]:
def display_wrong_result(pred_list, gold_list, dataset):
    for ind, (pred, gold) in enumerate(zip(pred_list, gold_list)):
        pred, gold = extract_yes_no_and_map(pred), extract_yes_no_and_map(gold)
        if pred != gold:
            sample= dataset[ind]
            df = pd.DataFrame(columns=sample["table"]["header"])
            for i, line in enumerate(sample['table']['rows']):
                df.loc[i] = line
            print(sample['statement'], sample['label'])
            print(tabulate(df, headers=df.columns, tablefmt='psql'))
        
print(display_wrong_result(batch_pred, ground, tabfact['validation']))

süper lig be the most common league to win a round in the turkish cup 1
+----+---------------+-------------------+------------------+-------------------------------+--------------------------+----------------------------------------------------+
|    | round         |   clubs remaining |   clubs involved | winners from previous round   | new entries this round   | leagues entering at this round                     |
|----+---------------+-------------------+------------------+-------------------------------+--------------------------+----------------------------------------------------|
|  0 | first round   |               156 |               86 | none                          | 86                       | tff third league & turkish regional amateur league |
|  1 | second round  |               113 |              108 | 43                            | 65                       | süper lig & tff first league & tff second league   |
|  2 | third round   |                59 |               5

In [40]:
for ind, i in enumerate(zip(batch_pred, ground)):
    print(ind, i)

0 ('1', '1')
1 ('1', '1')
2 ('1', '1')
3 ('1', '1')
4 ('1', '1')
5 ('1', '0')
6 ('0', '0')
7 ('0', '0')
8 ('0', '0')
9 ('0', '0')
10 ('1', '1')
11 ('0', '1')
12 ('1', '1')
13 ('1', '1')
14 ('0', '1')
15 ('0', '0')
16 ('0', '0')
17 ('0', '0')
18 ('1', '0')
19 ('1', '0')
20 ('The answer is 1.', '1')
21 ('0', '1')
22 ('1', '1')
23 ('0', '1')
24 ('1', '1')
25 ('0', '0')
26 ('0', '0')
27 ('1', '0')
28 ('0', '0')
29 ('0', '0')
30 ('1', '1')
31 ('1', '1')


## Answer output

In [1]:
preds = []
import json
with open('./result/data/tabfact_test_04-08_02-43-03.json', 'r') as f:
    lines = f.readlines()
    for l in lines:
        preds.append(json.loads(l))
    print(len(preds))
    
from data_loader import TableLoader

table_loader = TableLoader(table_name='tabfact', split='test', use_sample=True, small_test=True)

32


In [13]:
import re
matches = re.finditer(r'([^<]*)<([^\s>]*)>', preds[i]['pred'].split(':')[1])
for match in matches:
    print(match.group(1).strip())
    print(match.group(2))

year
KEEP
best teenage / young adult
DELETE
reader's vote
DELETE
best non-fiction
DELETE
lifetime achievement
KEEP


In [30]:
import pandas as pd
aug_data = pd.read_csv('./result/aug/tabfact_test_summary_31.csv', index_col='table_id')
print(aug_data.loc['2-18622227-6.html.csv']['column_description'])

tournament<The name of the tennis tournament>
1986<The performance of Katrina Adams in the tournament in 1986>
1988<The performance of Katrina Adams in the tournament in 1988>
1989<The performance of Katrina Adams in the tournament in 1989>
1990<The performance of Katrina Adams in the tournament in 1990>
1991<The performance of Katrina Adams in the tournament in 1991>
1992<The performance of Katrina Adams in the tournament in 1992>
1993<The performance of Katrina Adams in the tournament in 1993>
1994<The performance of Katrina Adams in the tournament in 1994>
1995<The performance of Katrina Adams in the tournament in 1995>
1996<The performance of Katrina Adams in the tournament in 1996>
1997<The performance of Katrina Adams in the tournament in 1997>
1998<The performance of Katrina Adams in the tournament in 1998>
1999<The performance of Katrina Adams in the tournament in 1999>


In [18]:
from data_loader import TableFormat
from executor import SQLManager
from sqlalchemy import create_engine
engine = create_engine('sqlite:///db/sqlite/tabfact.db', echo=False)
manager = SQLManager(engine)
i = 0
# SQL = manager.format_sql(preds[i]['pred'].split(':')[1])


def show_table(data):
    formatter = TableFormat(format='none', data=data)
    columns = [c.strip() for c in preds[i]['pred'].split(',')]
    print(data['statement'])
    formatter.data = formatter.data.loc[:, columns]
    
    print(formatter.format_html())
    print(formatter.format_psql())
    print(data['label'])
    # data.columns = [manager.normalize_col_name(c) for c in formatter.all_data.columns]
    # data.to_sql('DF', manager.engine, if_exists='replace', index=False)
    
    # subtable = pd.read_sql(command, self.engine)
    # test_df = manager.execute_from_df(SQL, formatter.all_data)
    # return test_df
test_df = show_table(table_loader.dataset[i])
print(test_df)

the 31997 mintage be release in 2001
<table>
<thead>
<tr><th>  year</th><th>                  mintage</th></tr>
</thead>
<tbody>
<tr><td>2003  </td><td>included in hmcs bras dor</td></tr>
<tr><td>2003  </td><td>31997                    </td></tr>
<tr><td>2000  </td><td>44367                    </td></tr>
</tbody>
</table>
+--------+----------------------------+-------------------+-----------------------------+---------------+
|   year | theme                      | artist            | mintage                     |   issue price |
|--------+----------------------------+-------------------+-----------------------------+---------------|
|   2000 | steam buggy                | john mardon       | 44367                       |         59.95 |
|   2000 | the bluenose               | j franklin wright | included in steam buggy     |         59.95 |
|   2000 | the toronto                | john mardon       | included in steam buggy     |         59.95 |
|   2001 | the russell light four     | 

In [4]:
preds[i]


{'table_name': '2-15753390-2.html.csv',
 'label': '2 be the fewest point that roger dutton / tony wright receive',
 'pred': 'rider, points'}

In [10]:

from data_loader import TableFormat
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.messages import HumanMessage
from langchain.prompts.prompt import PromptTemplate
model = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_base="https://api.chatanywhere.cn/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")
inds = [0, 100]
Output_examples = ['event name(KEEP) established(KEEP) category(KEEP) sub category(DELETE) main venue(DELETE)', 'wind farm, scheduled, capacity (mw), turbines, type, and location.']



pre_instruction = PromptTemplate(input_variables=["table", "claim"], template=
"""
Below are some rows from the sub-table filtered by an SQL query. You are required to infer the data distribution and format from the sample data of the sub-table. Carefully analyze the issue and, based on the problem, write an SQL statement that can provide the answer.
sub-table: {table}
Claim: {claim}
SQL: {sql}
""")
# 
output = model.invoke([HumanMessage(content=pre_instruction.format(table=formatter.format_html(), 
                                                                   claim="verify whether the provided claim/query are true or false. Return 0 if it's false, or 1 if it's true. Only return 0 or 1 without any other information. "+test_sample['statement'],
                                                                   sql = ' '.join(["SELECT event_name, category, established from tmp;","SELECT * FROM tmp\n WHERE event_name = 'touchdown atlantic' AND category = 'sporting' AND established = 2010;"])
                                                                   
                                                                   ))])



In [13]:
test_sample['label']

1

In [12]:
output.content

"SELECT * FROM table\nWHERE event_name = 'touchdown atlantic' AND category = 'sporting' AND established = 2010;"

In [ ]:
# Answer output
pre_instruction = PromptTemplate(input_variables=["table", "claim"], template=
"""
Our ultimate goal is to answer questions based on the table. Below is a filtered subset of the table. You need to understand the logic behind the SQL filtering and use the final subset to answer questions about the table.
sub-table: {table}
SQL Excuted: {SQL}
Claim: {}
""")
output = model.invoke([HumanMessage(content=pre_instruction.format(table=formatter.format_html(), claim=test_sample['statement']))])

## Evaluat from local result

In [17]:
file_name = 'result/data/tabfact_validation_04-04_01-51-14.json'
import json
from executor import SQLManager
data = []
with open(file_name, 'r') as f:
    lines =  f.readlines()
    for l in lines:
        data.append(json.loads(l))
print(len(data))


8


In [19]:
data[0]['predict']

'Output: round<DELETE> clubs remaining<DELETE> clubs involved<DELETE> winners from previous round<DELETE> new entries this round<KEEP> leagues entering at this round<DELETE>'

In [24]:
table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=False)
formatter = TableFormat(
format='none', data=table_loader.dataset[0], use_sampling=True)
engine = create_engine('sqlite:///db/sqlite/tabfact.db', echo=True)
formatter.all_data.to_sql('DF', engine, if_exists='replace', index=False)

manager = SQLManager(engine=engine)


2024-04-04 02:23:43,671 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-04 02:23:43,673 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("DF")
2024-04-04 02:23:43,674 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-04 02:23:43,678 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("DF")
2024-04-04 02:23:43,679 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-04 02:23:43,681 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-04-04 02:23:43,682 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-04 02:23:43,685 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-04-04 02:23:43,686 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-04 02:23:43,687 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("DF")
2024-04-04 02:23:43,688 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-04 02:23:43,689 INF

2024-04-04 02:23:43,694 INFO sqlalchemy.engine.Engine PRAGMA main.index_list("DF")
2024-04-04 02:23:43,694 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-04 02:23:43,695 INFO sqlalchemy.engine.Engine PRAGMA temp.index_list("DF")
2024-04-04 02:23:43,695 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-04 02:23:43,695 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("DF")
2024-04-04 02:23:43,696 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-04 02:23:43,696 INFO sqlalchemy.engine.Engine PRAGMA main.index_list("DF")
2024-04-04 02:23:43,697 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-04 02:23:43,697 INFO sqlalchemy.engine.Engine PRAGMA temp.index_list("DF")
2024-04-04 02:23:43,697 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-04 02:23:43,697 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("DF")
2024-04-04 02:23:43,698 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-04 02:23:43,698 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master 

In [26]:
table_name = 'DF'
E_sql = manager.format_sql(data[0]['predict'], table_name=table_name)
print(E_sql)
manager.run(E_sql)

SELECT new entries this round FROM DF
SQL:SELECT new entries this round FROM DF, ttype:Token.Keyword.DML, sql_type:SELECT, table:None
Query[SELECT new entries this round FROM DF]
2024-04-04 02:24:51,053 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-04 02:24:51,056 INFO sqlalchemy.engine.Engine SELECT new entries this round FROM DF
2024-04-04 02:24:51,058 INFO sqlalchemy.engine.Engine [generated in 0.00249s] ()


OperationalError: (sqlite3.OperationalError) near "this": syntax error
[SQL: SELECT new entries this round FROM DF]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [3]:
def split_answer(answer: str):
    specail_tokens = ['Output']
    for s in specail_tokens:
        if s in answer:
            parts = answer.split(s)
            break
    summary = parts[0].split(':')[1].strip()
    operations = parts[1].split(':')[1].strip()

    return summary, operations

In [9]:
table_loader.dataset[0]

{'id': 0,
 'table': {'id': '2-1859269-1.html.csv',
  'header': ['round',
   'clubs remaining',
   'clubs involved',
   'winners from previous round',
   'new entries this round',
   'leagues entering at this round'],
  'rows': [['first round',
    '156',
    '86',
    'none',
    '86',
    'tff third league & turkish regional amateur league'],
   ['second round',
    '113',
    '108',
    '43',
    '65',
    'süper lig & tff first league & tff second league'],
   ['third round', '59', '54', '54', 'none', 'none'],
   ['fourth round', '32', '32', '27', '5', 'süper lig'],
   ['fifth round', '16', '16', '16', 'none', 'none'],
   ['group stage', '8', '8', '8', 'none', 'none'],
   ['semi - finals', '4', '4', '4', 'none', 'none'],
   ['final', '2', '2', '2', 'none', 'none']],
  'caption': 'turkish cup'},
 'statement': 'during the third round of the turkish cup , there be no new entry during that stage',
 'label': 1,
 'hardness': 'complex',
 'small_test': False}

In [7]:
from executor import SQLManager
from sqlalchemy import create_engine
from data_loader import TableLoader, TableFormat
table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True)
print(table_loader.dataset[0])
def show_table(data):
    formatter = TableFormat(format='none', data=data)
    print(data['statement'])
    print(formatter.format_psql())
    print(data['label'])
    # sum, ope = split_answer(data[ind]['predict'])
    # print(ope)
    # print(manager.format_sql(ope, table_name='DF'))
show_table(table_loader.dataset[6]) 

{'id': 1260, 'table': {'id': '2-14101606-2.html.csv', 'header': ['wind farm', 'scheduled', 'capacity (mw)', 'turbines', 'type', 'location'], 'rows': [['codling', 'unknown', '1100', '220', 'unknown', 'county wicklow'], ['carrowleagh', '2012', '36.8', '16', 'enercon e - 70 2.3', 'county cork'], ['dublin array', '2015', '364', '145', 'unknown', 'county dublin'], ['glenmore', '2009 summer', '30', '10', 'vestas v90', 'county clare'], ['glenough', '2010 winter', '32.5', '13', 'nordex n80 / n90', 'county tipperary'], ['gortahile', '2010 autumn', '20', '8', 'nordex n90', 'county laois'], ['grouse lodge', '2011 summer', '20', '8', 'nordex n90', 'county tipperary'], ['moneypoint', 'unknown', '22.5', '9', 'unknown', 'county clare'], ['mount callan', 'unknown', '90', '30', '3 mw', 'county clare'], ['oriel', '2013', '330', '55', 'unknown', 'county louth'], ['skerd rocks', 'unknown', '100', '20', '5 mw', 'county galway'], ['shragh', 'planning submitted oct 2011', '135', '45', 'enercon e82 3.0 mw', '

In [16]:
show_table(5)

there be new entry for the 1st 4 round of the turkish cup
+---------------+-------------------+------------------+-------------------------------+--------------------------+----------------------------------------------------+
| round         |   clubs remaining |   clubs involved | winners from previous round   | new entries this round   | leagues entering at this round                     |
|---------------+-------------------+------------------+-------------------------------+--------------------------+----------------------------------------------------|
| first round   |               156 |               86 | none                          | 86                       | tff third league & turkish regional amateur league |
| second round  |               113 |              108 | 43                            | 65                       | süper lig & tff first league & tff second league   |
| third round   |                59 |               54 | 54                            | none    